# DOLPHOT/WFPC2

Notebook to do photometry with DOLPHOT and WFPC2 data. It calls the bash routines and the outpout redirected to a file so the notebook doesn't get too long.,

Dolphot2 doesnt per image some parameters and does it for all images

In [2]:
import glob
import os
import subprocess
from shutil import copy2
from dolphot2 import DolphotParameters

In [3]:
pwd

'/home/mmarcano/Documents/NGC6397F336W/u9712005F336ByChip'

In [4]:
datadir = 'Try/'


In [3]:
listainitials = glob.glob('Originals/u971*c[0-1]m.fits')
#drzinitial = glob.glob('Originals/u971010*drz.fits')
#drzinitialfile =drzinitial[0]
#drzinitialfilename = drzinitialfile.split('.')[0].split('/')[1]
#listainitials.append(drzinitial[0])
#print(listainitials)
finaldrizorfile = 'Driz/final_drz_sci.fits'
listainitials.append(finaldrizorfile)
drzfilename = finaldrizorfile.split('.')[0].split('/')[1]
print(drzfilename)
#print(listainitials)

for i in listainitials:
    copy2(i,datadir)

final_drz_sci


In [4]:
pattern = 'u*c0m.fits'
listacom = glob.glob('{}{}'.format(datadir,pattern))
listafits = glob.glob('{}{}'.format(datadir,'*fits'))


listacom

In [5]:
for i in listacom:
    t = i.replace('c0m','c1m')
    commandmask = 'wfpc2mask {} {} &>> wfpc2mask.log'.format(i,t)
    #print(commandmask)
    ! {commandmask}
! head wfpc2mask.log

Reading FITS file Try/u9712306m_c0m.fits
Reading IMAGE extension: 800x800
Reading IMAGE extension: 800x800
Reading IMAGE extension: 800x800
Reading IMAGE extension: 800x800
Reading FITS file Try/u9712306m_c1m.fits
Reading IMAGE extension: 800x800
Reading IMAGE extension: 800x800
Reading IMAGE extension: 800x800
Reading IMAGE extension: 800x800


# 3.3 splitgroups: Split UVIS Chips

In [6]:
dirfits = datadir+'*.fits'
#print(dirfits)
commandsplit = 'splitgroups {} &>> splitgroups.log'.format(dirfits)
#print(commandsplit)
! {commandsplit}
! head splitgroups.log

Reading FITS file Try/final_drz_sci.fits: 1549x1527
Reading BINTABLE extension: 2307x504
Writing FITS file Try/final_drz_sci.chip0.fits: 1549x1527
Writing FITS file Try/final_drz_sci.chip1.fits: 2307x504
Reading FITS file Try/u9710103m_c0m.fits
Reading IMAGE extension: 800x800
Reading IMAGE extension: 800x800
Reading IMAGE extension: 800x800
Reading IMAGE extension: 800x800
Writing FITS file Try/u9710103m_c0m.chip1.fits: 800x800


# 3.4 calcsky: Calculate Sky for Image

In [7]:
rin=10
rout = 25
step = 2
sigmalow = 2.25
sigmahigh = 2.00
listacomall = glob.glob('{}{}'.format(datadir,'u971*m_c0m.*.fits'))
listachipdrz = glob.glob('{}{}'.format(datadir,drzfilename+'*chip*'))
listacomall.extend(listachipdrz)
#listafits = glob.glob('{}{}'.format(datadir,'*fits'))
#print(listachipdrz)
for i in listacomall:
    name = i.split('.fits')[0]
    #print(name)
    #name = 'Try/u9710103m_c0m.chip1'
    calcskycommad = 'calcsky {} {} {} {} {} {} &>> calcsky.log'.format(name,rin,rout,step,sigmalow, sigmahigh)
    #print(calcskycommad)
    #! {calcskycommad}

#! head calcsky.log


# Build Param file

This is a great exampleto build a param file in htis git repo:

https://github.com/jonathansick/delphinus

Ill use this to build the params file.


Do it only one or it will add

In [5]:
dolpa = DolphotParameters(psfA=(3, 0, 0, 0, 0, 0),
            psfB=(3, 0, 0, 0, 0, 0), psfC=(0, 0, 0, 0, 0, 0),
            shift=(0, 0), xform=(1, 0, 0), aprad=0.5, apsky=(15, 25),
            RAper=7, RSky2=(5.0, 10.0), RChi=2, RSky=(15.0, 35.0) , RPSF=13,
            PSFPhot=1, FitSky=2, SkipSky=2, SkySig=2.25,
            SigFind=2.5, SigFindMult=0.85,
            SigFinal=3.0, MaxIT=25, NoiseMult=0.10, FSat=0.999, Zero=25.0,
            RCentroid=2, PosStep=0.25,
            dPosMax=3.0, RCombine=1.5, FPSF="G+L",
            sigPSF=5.0, PSFStep=0.25, MinS=1.0, MaxS=9.0, MaxE=0.5,
            UseWCS=1, Align=2, Rotate=1, secondPass=5, SearchMode=1,
            Force1=0,
            EPSF=1,
            PSFsol=0, PSFres=1, psfstars=None, psfoff=0.0,
            ApCor=1, SubPixel=1,
            FakeStars=None, FakeMatch=3., FakeStarPSF=0, FakePSF=1.5,
            RandomFake=1,
            xytfile=None, xytpsf=None, photsec=None,
                WFPC2useCTE=1,FlagMask=4)



In [6]:
dolpa.imageParams

[]

# Try chip3

In [7]:
#dolpa.setup_image('Try/final_drz_sci.chip0',ref=True)

In [8]:
dolpa.setup_image('Try/u9710103m_c0m.chiptres',ref=True)

In [9]:
#listacomallchip3 = glob.glob('{}{}'.format(datadir,'u971*[!103]m_c0m.chip3.fits'))
listacomallchip = glob.glob('{}{}'.format(datadir,'u971*m_c0m.chip[3].fits'))

#Try with subset
listacomallchip = listacomallchip

for i in listacomallchip:
    dolpa.setup_image(i)
    #print(i)

In [10]:
dolpa.write_parameters('dolphot.param')

In [11]:
!head dolphot.param

Nimg = 125
img0_file = Try/u9710103m_c0m.chiptres
img1_file = Try/u9710106m_c0m.chip3
img2_file = Try/u9710109m_c0m.chip3
img3_file = Try/u9710203m_c0m.chip3
img4_file = Try/u9710206m_c0m.chip3
img5_file = Try/u9710209m_c0m.chip3
img6_file = Try/u9710303m_c0m.chip3
img7_file = Try/u9710306m_c0m.chip3
img8_file = Try/u9710403m_c0m.chip3


In [12]:
dolpa.params

{'img_psfa': (3, 0, 0, 0, 0, 0),
 'img_psfb': (3, 0, 0, 0, 0, 0),
 'img_psfc': (0, 0, 0, 0, 0, 0),
 'img_shift': (0, 0),
 'img_xform': (1, 0, 0),
 'img_aprad': 0.5,
 'img_apsky': (15, 25),
 'img_RSky': (15.0, 35.0),
 'img_RSky2': (5.0, 10.0),
 'img_RAper': 7,
 'img_RChi': 2,
 'img_RPSF': 13,
 'PSFPhot': 1,
 'FitSky': 2,
 'SkipSky': 2,
 'SkySig': 2.25,
 'SigFind': 2.5,
 'SigFindMult': 0.85,
 'SigFinal': 3.0,
 'MaxIT': 25,
 'NoiseMult': 0.1,
 'FSat': 0.999,
 'Zero': 25.0,
 'RCentroid': 2,
 'PosStep': 0.25,
 'dPosMax': 3.0,
 'RCombine': 1.5,
 'FPSF': 'G+L',
 'sigPSF': 5.0,
 'PSFStep': 0.25,
 'MinS': 1.0,
 'MaxS': 9.0,
 'MaxE': 0.5,
 'UseWCS': 1,
 'Align': 2,
 'Rotate': 1,
 'secondPass': 5,
 'SearchMode': 1,
 'Force1': 0,
 'EPSF': 1,
 'PSFsol': 0,
 'PSFres': 1,
 'psfstars': None,
 'psfoff': 0.0,
 'ApCor': 1,
 'SubPixel': 1,
 'FakeStars': None,
 'FakeMatch': 3.0,
 'FakeStarPSF': 0,
 'FakePSF': 1.5,
 'RandomFake': 1,
 'xytfile': None,
 'xytpsf': None,
 'photsec': None,
 'WFPC2useCTE': 1,
 'F